# 42-create-kmodes-final-dataframe
> In this notebook, I am going to create 2 dataframes which will be used for calculating Euclidean distance in the next netebook 50.

There are 4 methods we are planning:

1. including ['votes'] column + Kmode + Kmeans
2. including ['votes'] column + Kmode 
3. NOT including ['votes'] column + Kmode + Kmeans
4. NOT including ['votes'] column + Kmode 

In this notebook, I am going to create 2 and 4.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Import data

For the first step, we need to import data. If you want to run this code in your local computer, please change the path of "personal_path" corresponding to your local environment.

In [3]:
# Define file paths
prefix_path = os.path.expanduser('~/')
# If you want to run this code, please edit personal path below.
personal_path = 'Vanderbilt University/2021 Fall/'
folder_path = 'case2/Data/cluster16_kmode/'
path = prefix_path + personal_path + folder_path

# Import all csv file in the cluster16_kmode folder.
csv_files = glob.glob(path + "*.csv")

Let's import all csv files and save this in "df_list". By doing this, we can call every csv files quickly and it helps us handle many data easily.

In [4]:
# Import dataframe and assign them to a list
df_list = []
for i in range(len(csv_files)):
    df_list.append(pd.read_csv(csv_files[i]))

let's create a new column "group". This value is refers to a group number after we diveded the data using Kmode in the previous notebook 40.

In [5]:
for i in range(len(df_list)):
    df_list[i]['group'] = i + 1

Now, let's combine all dataframe. The reason why we are combining all data together is because we are going to normalize data. 

In [6]:
# Merging multiple dataframes
df_total = [df_list[0], df_list[1],df_list[2], df_list[3], df_list[4], df_list[5], df_list[6], df_list[7], df_list[8], df_list[10], df_list[11], df_list[12]]
all_df = pd.concat(df_total)

# Pre-processing dataframe

When we clustering, we do not need 'recommendation1','recommendation2','recommendation3','recommendation4','recommendation5','recommendation6' columns. Let's remove these 6 columns.

In [7]:
all_df = all_df.drop(['recommendation1','recommendation2','recommendation3','recommendation4','recommendation5','recommendation6'], axis = 1)

Also, in the previous notebook 41, we found that there were 14 movies having no values. Let's remove this movies as well.

In [8]:
no_values_id = ['tt0246425',
 'tt0442781',
 'tt0043852',
 'tt0906783',
 'tt0103790',
 'tt0370919',
 'tt0238119',
 'tt0033368',
 'tt0100100',
 'tt0253614',
 'tt0142634',
 'tt0082892',
 'tt0131636',
 'tt0276868']

In [9]:
final_df_temp = all_df[~all_df["id"].isin(no_values_id)]

By not using some groups, the group's number is messed. Let's reorganize this.

In [10]:
final_df_temp["group"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13])

In [11]:
final_df_temp.loc[final_df_temp["group"] == 11, "group"] = 10
final_df_temp.loc[final_df_temp["group"] == 12, "group"] = 11
final_df_temp.loc[final_df_temp["group"] == 13, "group"] = 12

In [13]:
final_df_temp.head()

,id,review_1st,review_2nd,description_1st,description_2nd,overview_1st,overview_2nd,votes,reviews_from_critics,popularity,group
0,tt0248123,5.052363,14.008677,19.101181,-5.684755,10.454543,5.274888,6.60,7.0,0.041677,1
1,tt0443584,0.486805,11.690599,20.577621,-4.113286,13.450911,1.172032,4.45,10.0,5.257142,1
2,tt0242527,2.242022,11.166927,21.120935,-2.160072,9.920180,0.142537,6.30,47.0,4.406514,1
3,tt0374180,5.129464,9.997657,18.254795,-2.657217,10.732504,1.921588,5.35,33.0,8.643620,1
4,tt0428856,4.142411,12.404982,19.261920,-5.632376,12.010869,2.989712,6.40,40.0,5.545420,1


# Noramlization

Similarity is based on the distance between data point's features. For this reason, normalization is very important for calculating distance. I am going to use min-max normalization to each column. After this process, values will be between 0 to 1.

In [14]:
def normalize(df):
    """
    Normalize all values in dataframe
    
    Parameters
    ----------
    df : dataframe
        a dataframe we want to normalize
        
    Returns
    -------
    df 
        a normalized dataframe
    """
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [15]:
final_df_temp_id = final_df_temp[['id', 'group']]
left_over = final_df_temp.drop(['id', 'group'], axis = 1)

In [16]:
# Perform normalization
df_norm = normalize(left_over)

In [17]:
df_norm.head()

,review_1st,review_2nd,description_1st,description_2nd,overview_1st,overview_2nd,votes,reviews_from_critics,popularity
0,0.809561,0.784496,0.450726,0.026841,0.841725,0.437456,0.738854,0.008197,0.000225
1,0.119524,0.424670,0.662225,0.261958,0.978468,0.143147,0.464968,0.012295,0.028406
2,0.384807,0.343383,0.740054,0.554190,0.817338,0.069299,0.700637,0.062842,0.023810
3,0.821214,0.161882,0.329482,0.479809,0.854410,0.196915,0.579618,0.043716,0.046704
4,0.672031,0.535561,0.473752,0.034677,0.912749,0.273534,0.713376,0.053279,0.029964


In [18]:
df_final = pd.concat([final_df_temp_id, df_norm], axis = 1)

In [20]:
df_final.head()

,id,group,review_1st,review_2nd,description_1st,description_2nd,overview_1st,overview_2nd,votes,reviews_from_critics,popularity
0,tt0248123,1,0.809561,0.784496,0.450726,0.026841,0.841725,0.437456,0.738854,0.008197,0.000225
1,tt0443584,1,0.119524,0.424670,0.662225,0.261958,0.978468,0.143147,0.464968,0.012295,0.028406
2,tt0242527,1,0.384807,0.343383,0.740054,0.554190,0.817338,0.069299,0.700637,0.062842,0.023810
3,tt0374180,1,0.821214,0.161882,0.329482,0.479809,0.854410,0.196915,0.579618,0.043716,0.046704
4,tt0428856,1,0.672031,0.535561,0.473752,0.034677,0.912749,0.273534,0.713376,0.053279,0.029964


Great! We just created "2. including ['votes'] column + Kmode" dataframe.

To create "4. NOT including ['votes'] column + Kmode" dataframe, let's remove 'votes' column.

In [22]:
df_final_no_votes = df_final.drop(['votes'], axis = 1)

In [23]:
df_final_no_votes.head()

,id,group,review_1st,review_2nd,description_1st,description_2nd,overview_1st,overview_2nd,reviews_from_critics,popularity
0,tt0248123,1,0.809561,0.784496,0.450726,0.026841,0.841725,0.437456,0.008197,0.000225
1,tt0443584,1,0.119524,0.424670,0.662225,0.261958,0.978468,0.143147,0.012295,0.028406
2,tt0242527,1,0.384807,0.343383,0.740054,0.554190,0.817338,0.069299,0.062842,0.023810
3,tt0374180,1,0.821214,0.161882,0.329482,0.479809,0.854410,0.196915,0.043716,0.046704
4,tt0428856,1,0.672031,0.535561,0.473752,0.034677,0.912749,0.273534,0.053279,0.029964


# Save each dataframe to csv files

In [21]:
df_final.to_csv(prefix_path + personal_path + 'case2/Data/yesVotes_kmode_final_df.csv')

In [24]:
df_final_no_votes.to_csv(prefix_path + personal_path + 'case2/Data/noVotes_kmode_final_df.csv')